# Question

We ask the question: If the states $\rho$ and $\sigma$ are close to each other $\Vert \rho - \sigma \Vert_1 \leq \epsilon$, does there exist a projector $\Pi$, such that $\Pi \rho \Pi \leq (1+g_1(\epsilon)) \sigma$ and $\text{tr}(\Pi \sigma) \geq 1- g_2(\epsilon)$ for some small functions $g_1(\epsilon)$ and $g_2(\epsilon)$?

In this notebook, we focus on the case where the projectors consist of the eigenvectors of $\sigma$. 

Package imports:

In [148]:
import numpy as np
import random as rnd
from pprint import pprint
from numpy.random import default_rng


# Importing stuff to make random sampling of psd matrices possible
import sys
sys.path.append('/Users/vivekmarwah/Documents/Research scripts')
import PSDMatrixFunctions as psdFn


We define the functions as $g_1(\epsilon):= \epsilon^{\alpha}$ and $g_2(\epsilon):= \epsilon^{\beta}$ for $\alpha + \beta \leq 1$, which is what works in the 2 dim case.

<bold>Choice of $\alpha, \beta$ </bold>: We can't choose $g_1$ and $g_2$ to be too small, since the maximum N we can choose is about ~40. 

I think a dimension dependent g1 could always work, so we want say $g_2$*N to still be small.


In [149]:
def g1(eps):
    return np.power(eps,0.5)

def g2(eps):
    return np.power(eps,0.5)

Parameters for the problem:
- N: dimension of $\rho$ and $\sigma$
- eps_max: is the maximum possible $\epsilon$ in the problem statement, we calculate actual distance later

### Construction of $\sigma$

We can assume WLOG that $\sigma$ is a diagonal matrix. We need to make sure that at least some eigenvalues of $\sigma$ are smaller than $g_2(\epsilon)$, otherwise we can prove that the statement is true.

num_small: represents number of eigenvalues smaller than $g_2(\epsilon)$

In [150]:
def sample_sigma(N: int, eps_max: float) -> np.ndarray:
    '''Samples a random diagonal matrix sigma'''
    num_small= rnd.randint(1, N-1)
    eig_sigma= []

    # Select num_small numbers smaller than g2(eps_max) randomly
    for i in range(num_small):
        eig_sigma.append(rnd.uniform(0, g2(eps_max)))

    # Calculate the sum of all the small eigenvalues
    sum_small = sum(eig_sigma)

    # Ensure that the sum of the small eigenvalues does not exceed 1
    if sum_small > 1: 
        target_sum_small = rnd.uniform(0,1)

        # Rescale the small eigenvalues so that their sum is 
        eig_sigma = list(map(
                        lambda x: x*target_sum_small/sum_small, 
                        eig_sigma
                        )
                    )
        sum_small = target_sum_small

    # Choose the rest of the eigenvalues
    eig_sigma_large = []

    for i in range(num_small, N): 
        eig_sigma_large.append(rnd.uniform(0,1))

    sum_large = sum(eig_sigma_large)
    target_sum_large = 1- sum_small

    # Rescale them so that their sum is 1- sum_small
    eig_sigma_large = list(map(
                        lambda x: x*target_sum_large/sum_large, 
                        eig_sigma_large
                        )
                    )

    eig_sigma.extend(eig_sigma_large)

    # Put the eigenvalues constructed in a diagonal matrix to create sigma 
    sigma = np.diag(eig_sigma)
    return sigma

### Sampling $\rho$

To sample $\rho$, we sample two PSD matrices with trace=1, $P$ and $Q$ randomly until the matrix 
$$\sigma + \frac{\epsilon}{2}P - \frac{\epsilon}{2}Q$$
is positive. We choose this positive matrix to be $\rho$. Clearly, this has trace=1 and $\Vert \rho - \sigma \Vert_1 \leq \epsilon_{\max}$. We calculate the actual distance and use it for computation.

In [151]:
rng = default_rng()

def sample_rho(N: int, eps_max: float, sigma: np.ndarray, num_tries: int = 100): 
    '''Samples a positive matrix rho of dimension [N] atmost [eps_max] far from sigma. 
    Probabilistic procedure runs only for [num_tries] and then throw an error'''
    while num_tries>0:
        num_tries = num_tries-1
        P= psdFn.randomPositiveMatrix(rng, N)
        P= P/np.trace(P)
        Q=psdFn.randomPositiveMatrix(rng, N)
        Q= Q/np.trace(Q)
        rho = np.array(sigma + eps_max/2*P - eps_max/2*Q)
        # Need to make sure that the tolerance of the function doesn't mess up the dist
        if psdFn.isPositive(rho, tol=eps_max*1e-4):
            return rho
    raise ValueError("Exhausted number of tries for given values")

The following function tests whether the rho and sigma sampling work right.

In [152]:
# Test for the above rho and sigma sampling
def test_sampling(N, eps_max, num_test):
    num_fail=0
    for i in range(num_test):
        sigma = sample_sigma(N, eps_max)
        try:
            rho = sample_rho(N, eps_max, sigma)
            # 'nuc' represents nuclear/ 1-norm
            dist = np.linalg.norm(sigma-rho, ord='nuc')
            if (dist > eps_max or
                abs(np.trace(rho) -1) > eps_max*1e-4 or
                not psdFn.isPositive(rho, tol=eps_max*1e-4)):
                print("Test Fail")
                print('Distance:', dist)
                print('trace of rho:', abs(np.trace(rho) -1))
                return
        except ValueError:
            num_fail+=1
    
    print("Test successful")
    print("num_fail= ", num_fail)

# Precision of numpy double (standard float for np) is only 16 digits
# test_sampling(100, 1e-11, 10000)

# Test above works for 
# N=100 and eps_max= 1e-11
# N=1000 and eps_max=1e-10


# Choosing the projector

Now we will create a function to go through all the possible projectors of $\sigma$. Note, we can restrict ourselves to projectors, which include the eigenvectors corresponding to the large eigenvalues. 

In [153]:
from itertools import combinations

# This function requires approximately 2^(N/2) time on average
def goodProjectorExists(N, sigma, rho) -> tuple[bool, np.ndarray]:
    '''Returns true or false depending on whether a good projector exists. 
    Also returns the projector if it exists'''
    dist = np.linalg.norm(sigma-rho, ord='nuc')
    # indices of small eig
    eig_small=[i for i in range(N) if sigma[i,i] < g2(dist)]
    # indices of large eig
    eig_large = [i for i in range(N) if i not in eig_small]
    # Number of loops here = 2^len(eig_small)
    # k cycles through all the possible sizes of subsets of eig_small
    for k in range(len(eig_small)+1):
        # comb cycles through k sized combinations of eig_small
        for comb in combinations(eig_small, k):
            # Fill up the projector correctly
            proj = np.zeros((N,N))
            for i in range(N):
                if i in comb or i in eig_large:
                    proj[i,i]=1
            # @ is the symbol for matrix multiplication
            proj_rho = proj @ rho @ proj
            diff = (1+g1(dist))*sigma - proj_rho
            if psdFn.isPositive(diff, tol=dist*1e-4):
                return (True, proj)
    return (False, None)
            

def test_goodProjectorExists(N, eps_max, num_trial):
    for i in range(num_trial):
        sigma =sample_sigma(N, eps_max)
        rho = sample_rho(N, eps_max, sigma)
        result, _ = goodProjectorExists(N, sigma, rho)
        if not result: 
            print("Failure")
            print(sigma)
            print(rho)
            return
    print("Test successful")

# This test always has to be successful, since the statement is true for N=2
# test_goodProjectorExists(2, 1e-10, 1000)

# This is successful
test_goodProjectorExists(40, 1e-11, 10000)

# N=4
# eps_max=1e-10
# sigma =sample_sigma(N, eps_max)
# rho = sample_rho(N, eps_max, sigma)
# result, proj = goodProjectorExists(N, sigma, rho)

# pprint(sigma)
# pprint(rho)
# print(result)
# pprint(proj)


Test successful
